In [1]:
import pandas as pd
import numpy as np

# Define the number of rows
num_rows = 1000

# Create a DataFrame with random data
df = pd.DataFrame({
    'A': np.random.rand(num_rows),
    'B': np.random.rand(num_rows),
    'C': np.random.rand(num_rows),
    'D': np.random.rand(num_rows)
})

# Display the first few rows of the DataFrame
display(df)

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e1cd200c-4843-45f5-afc1-2f8378dcbfb7)

In [2]:
sdf = spark.createDataFrame(df)

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 4, Finished, Available, Finished)

In [3]:
delta_table_path = "Tables/random_table"
sdf.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(delta_table_path)

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 5, Finished, Available, Finished)

In [4]:
files = mssparkutils.fs.ls(delta_table_path)
files

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 6, Finished, Available, Finished)

[FileInfo(path=abfss://27c5a96b-2992-4efa-aaf0-703b017047c1@onelake.dfs.fabric.microsoft.com/b9d9a656-fbcf-4fe2-a8ff-6e68cd7ba4ce/Tables/random_table/_delta_log, name=_delta_log, size=0),
 FileInfo(path=abfss://27c5a96b-2992-4efa-aaf0-703b017047c1@onelake.dfs.fabric.microsoft.com/b9d9a656-fbcf-4fe2-a8ff-6e68cd7ba4ce/Tables/random_table/part-00000-22e5716c-23e3-44fe-90ee-a9a858a5c10c-c000.snappy.parquet, name=part-00000-22e5716c-23e3-44fe-90ee-a9a858a5c10c-c000.snappy.parquet, size=38885),
 FileInfo(path=abfss://27c5a96b-2992-4efa-aaf0-703b017047c1@onelake.dfs.fabric.microsoft.com/b9d9a656-fbcf-4fe2-a8ff-6e68cd7ba4ce/Tables/random_table/part-00000-49dad3cc-cca1-46e5-b8a4-9951d9bc01cd-c000.snappy.parquet, name=part-00000-49dad3cc-cca1-46e5-b8a4-9951d9bc01cd-c000.snappy.parquet, size=38885)]

In [5]:
delta_file = files[1]
mssparkutils.fs.rm(delta_file.path)

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 7, Finished, Available, Finished)

True

In [6]:
mssparkutils.fs.ls(delta_table_path)

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 8, Finished, Available, Finished)

[FileInfo(path=abfss://27c5a96b-2992-4efa-aaf0-703b017047c1@onelake.dfs.fabric.microsoft.com/b9d9a656-fbcf-4fe2-a8ff-6e68cd7ba4ce/Tables/random_table/_delta_log, name=_delta_log, size=0),
 FileInfo(path=abfss://27c5a96b-2992-4efa-aaf0-703b017047c1@onelake.dfs.fabric.microsoft.com/b9d9a656-fbcf-4fe2-a8ff-6e68cd7ba4ce/Tables/random_table/part-00000-49dad3cc-cca1-46e5-b8a4-9951d9bc01cd-c000.snappy.parquet, name=part-00000-49dad3cc-cca1-46e5-b8a4-9951d9bc01cd-c000.snappy.parquet, size=38885)]

In [8]:
def restore_files(file_api_path:str, workspace_name: str, lakehouse_name: str) -> list:
    """url = f"https://onelake.dfs.fabric.microsoft.com/{lakehouse_name}.Lakehouse/{file_api_path}" """
    from azure.storage.filedatalake import (
        DataLakeServiceClient,
        DataLakeDirectoryClient,
        FileSystemClient,
    )

    from azure.storage.blob import BlobServiceClient
    from azure.core.credentials import AccessToken, TokenCredential

    ACCOUNT_NAME = "onelake"
    data_path = f"{lakehouse_name}.Lakehouse/{file_api_path}"

    class CustomTokenCredential(TokenCredential):
        def __init__(self):
            self._token = mssparkutils.credentials.getToken("Storage")

        def get_token(self, *scopes, **kwargs):
            return AccessToken(self._token, float('inf'))

    # Example usage
    credential = CustomTokenCredential()

    #Create a service client using the default Azure credential

    account_url = f"https://{ACCOUNT_NAME}.dfs.fabric.microsoft.com"
    #token_credential = DefaultAzureCredential()

    service_client = DataLakeServiceClient(account_url, credential=credential)

    #Create a file system client for the workspace
    file_system_client = service_client.get_file_system_client(workspace_name)

    #List a directory within the filesystem
    paths = file_system_client.get_paths(path=data_path)
    deleted_paths = file_system_client.list_deleted_paths(path_prefix=data_path)

    bsc = BlobServiceClient(account_url, credential=credential)
    ccli = bsc.get_container_client(container=workspace_name)
    for path in deleted_paths:
        blob_client = ccli.get_blob_client(path)
        blob_client.undelete_blob()
    return deleted_paths

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 10, Finished, Available, Finished)

In [10]:
import sempy.fabric as fabric

workspace_id = fabric.get_workspace_id()
lakehouse_item_id = fabric.get_lakehouse_id()

workspace_name = "lakehouse_restore"
lakehouse_name = "restore_lh"
file_api_path = delta_table_path

files = restore_files(file_api_path=file_api_path, workspace_name=workspace_name, lakehouse_name=lakehouse_name)
for file in files:
    print("Restored: " + file)

StatementMeta(, 5e75fb55-3b33-4b27-b4cc-c244b3d25a44, 12, Finished, Available, Finished)

deleted:
